In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [8]:
df = pd.read_csv('data/merged_data_brand_dropna.csv')

In [9]:
# datetime 컬럼 변환
df['event_time_moscow'] = pd.to_datetime(df['event_time_moscow'])

### 조회 비율 높은 제품

In [15]:
import pandas as pd

# 1. 전체 유저 수
total_users = df['user_id'].nunique()

# 2. 제품별 조회 유저 수
product_view_users = df[df['event_type'] == 'view'].groupby('product_id')['user_id'].nunique().reset_index()
product_view_users.columns = ['product_id', 'view_user_count']

# 3. 유저 수 대비 조회 비율 계산
product_view_users['view_ratio'] = product_view_users['view_user_count'] / total_users * 100  # 퍼센트로 보기 좋게

# 4. 상위 제품 정렬
top_products = product_view_users.sort_values(by='view_ratio', ascending=False)

In [18]:
# 결과 확인
print(top_products.head(20))

       product_id  view_user_count  view_ratio
12681     5809910            53367    4.433893
12683     5809912            17780    1.477217
7922      5751383            15937    1.324094
24023     5877454            15716    1.305733
24025     5877456            15143    1.258127
7961      5751422            14787    1.228549
2938      5649236            13615    1.131176
18733     5849033            13326    1.107165
20256     5856186            13209    1.097444
12682     5809911            12325    1.023999
12348     5808664            10655    0.885250
14244     5816170            10307    0.856337
8679      5769877            10105    0.839554
18254     5846437             9536    0.792280
3687      5677043             8883    0.738027
18964     5850281             8549    0.710277
26235     5896420             8467    0.703464
5550      5700037             8196    0.680949
1952      5560754             8169    0.678705
10214     5785144             6945    0.577012


In [20]:
# CSV로 저장
top_products.head(10).to_csv('data/06_top_10_products_by_view_ratio.csv', index=False)
top_products.head(20).to_csv('data/06_top_20_products_by_view_ratio.csv', index=False)

In [19]:
import pandas as pd

# 1. event_time에서 월 정보 추출
df['event_time'] = pd.to_datetime(df['event_time_moscow'], errors='coerce')
df['event_month'] = df['event_time'].dt.to_period('M').astype(str)  # 'YYYY-MM' 형식

# 2. 결과 저장용 리스트
monthly_top_products = []

# 3. 월별 반복
for month in sorted(df['event_month'].dropna().unique()):
    month_df = df[df['event_month'] == month]
    
    # 월별 전체 유저 수
    total_users = month_df['user_id'].nunique()
    
    # 제품별 조회 유저 수
    product_view_users = (
        month_df[month_df['event_type'] == 'view']
        .groupby('product_id')['user_id']
        .nunique()
        .reset_index()
    )
    product_view_users.columns = ['product_id', 'view_user_count']
    
    # 유저 수 대비 조회 비율 계산
    product_view_users['view_ratio'] = product_view_users['view_user_count'] / total_users * 100
    
    # 월 정보 추가
    product_view_users['event_month'] = month
    
    # 상위 10개 제품만 추출
    top_10 = product_view_users.sort_values(by='view_ratio', ascending=False).head(10)
    
    # 누적
    monthly_top_products.append(top_10)

# 4. 하나의 데이터프레임으로 병합
top_10_by_month_df = pd.concat(monthly_top_products, ignore_index=True)

# 5. 확인 및 저장
print(top_10_by_month_df.head())
top_10_by_month_df.to_csv('data/06_monthly_top10_product_view_ratio.csv', index=False)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22204\383060568.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['event_month'] = df['event_time'].dt.to_period('M').astype(str)  # 'YYYY-MM' 형식


   product_id  view_user_count  view_ratio event_month
0     5809910             5147    1.805800     2019-10
1     5877454             3890    1.364788     2019-10
2     5751383             3383    1.186909     2019-10
3     5877456             3348    1.174630     2019-10
4     5751422             3097    1.086568     2019-10


- 브랜드별로, 월별 상위 10개

In [22]:
import pandas as pd

# 1. event_time → event_month 추출
df['event_time'] = pd.to_datetime(df['event_time_moscow'], errors='coerce')
df['event_month'] = df['event_time'].dt.to_period('M').astype(str)

# 2. 결과 저장 리스트
monthly_brand_top_products = []

# 3. 월별 반복
for month in sorted(df['event_month'].dropna().unique()):
    month_df = df[df['event_month'] == month]
    
    # 월별 전체 유저 수
    total_users = month_df['user_id'].nunique()

    # 브랜드별 반복
    for brand in month_df['brand'].dropna().unique():
        brand_df = month_df[(month_df['brand'] == brand) & (month_df['event_type'] == 'view')]

        if brand_df.empty:
            continue
        
        # 제품별 조회 유저 수
        product_view_users = (
            brand_df.groupby('product_id')['user_id']
            .nunique()
            .reset_index()
        )
        product_view_users.columns = ['product_id', 'view_user_count']

        # 유저 수 대비 조회 비율
        product_view_users['view_ratio'] = product_view_users['view_user_count'] / total_users * 100
        
        # 월/브랜드 정보 추가
        product_view_users['event_month'] = month
        product_view_users['brand'] = brand

        # 상위 10개만 추출
        top_10 = product_view_users.sort_values(by='view_ratio', ascending=False).head(10)
        
        # 누적
        monthly_brand_top_products.append(top_10)

# 4. 하나의 데이터프레임으로 병합
top_10_brand_by_month_df = pd.concat(monthly_brand_top_products, ignore_index=True)

# 5. 저장
top_10_brand_by_month_df.to_csv('data/06_monthly_top10_products_by_brand.csv', index=False)

# 6. 확인
print(top_10_brand_by_month_df.head())

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22204\2598065011.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['event_month'] = df['event_time'].dt.to_period('M').astype(str)


   product_id  view_user_count  view_ratio event_month   brand
0     5856186             2817    0.988331     2019-10  runail
1     5700037             1755    0.615733     2019-10  runail
2     5785144             1304    0.457502     2019-10  runail
3     5856191             1273    0.446626     2019-10  runail
4     5749720             1157    0.405928     2019-10  runail


In [23]:
top_10_brand_by_month_df

,product_id,view_user_count,view_ratio,event_month,brand
0,5856186,2817,0.988331,2019-10,runail
1,5700037,1755,0.615733,2019-10,runail
2,5785144,1304,0.457502,2019-10,runail
3,5856191,1273,0.446626,2019-10,runail
4,5749720,1157,0.405928,2019-10,runail
...,...,...,...,...,...
10474,5928238,22,0.007508,2020-02,resize
10475,5928235,15,0.005119,2020-02,resize
10476,5928233,12,0.004095,2020-02,resize
10477,5928237,6,0.002048,2020-02,resize


### 제품 종류 수

In [5]:
import pandas as pd

# event_time을 datetime으로 변환 후 월 추출
df['event_month'] = df['event_time_moscow'].dt.to_period('M').astype(str)

# 1. 전체 제품 종류 수
total_unique_products = df['product_id'].nunique()

# 2. 월별 제품 종류 수
monthly_unique_products = (
    df.groupby('event_month')['product_id']
    .nunique()
    .reset_index()
    .rename(columns={'product_id': 'unique_product_count'})
)

# 3. 전체 수 추가 (선택사항)
monthly_unique_products.loc[-1] = ['전체', total_unique_products]
monthly_unique_products = monthly_unique_products.sort_index().reset_index(drop=True)

# 결과 출력
print(monthly_unique_products)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22204\1988968411.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['event_month'] = df['event_time_moscow'].dt.to_period('M').astype(str)


  event_month  unique_product_count
0          전체                 31083
1     2019-10                 23973
2     2019-11                 24300
3     2019-12                 25675
4     2020-01                 25752
5     2020-02                 27254


In [6]:
# CSV 저장
monthly_unique_products.to_csv('data/12_monthly_unique_product_count.csv', index=False)

In [25]:
import pandas as pd

# event_time을 datetime으로 변환 후 월 추출
df['event_time'] = pd.to_datetime(df['event_time_moscow'], errors='coerce')
df['event_month'] = df['event_time'].dt.to_period('M').astype(str)

# 1. 전체 브랜드별 제품 종류 수
total_unique_by_brand = (
    df.groupby('brand')['product_id']
    .nunique()
    .reset_index()
    .rename(columns={'product_id': 'total_unique_products'})
)

# 2. 월별 브랜드별 제품 종류 수
monthly_unique_by_brand = (
    df.groupby(['event_month', 'brand'])['product_id']
    .nunique()
    .reset_index()
    .rename(columns={'product_id': 'unique_product_count'})
)

# 3. 전체 행 추가 (선택사항) — 월 대신 '전체'로 추가
total_unique_by_brand['event_month'] = '전체'
total_combined = pd.concat([monthly_unique_by_brand, total_unique_by_brand[['event_month', 'brand', 'total_unique_products']].rename(columns={'total_unique_products': 'unique_product_count'})])

# 정렬 (선택사항)
total_combined = total_combined.sort_values(['event_month', 'brand']).reset_index(drop=True)

# 결과 출력
print(total_combined.head(20))  # 일부만 출력

# CSV로 저장 (선택사항)
total_combined.to_csv('data/12_monthly_unique_products_by_brand.csv', index=False)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22204\3489210969.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['event_month'] = df['event_time'].dt.to_period('M').astype(str)


   event_month          brand  unique_product_count
0      2019-10       airnails                   218
1      2019-10          almea                    32
2      2019-10         andrea                     2
3      2019-10         ardell                    54
4      2019-10     art-visage                    61
5      2019-10          artex                    16
6      2019-10           aura                    10
7      2019-10      australis                    23
8      2019-10       balbcare                     2
9      2019-10         barbie                     3
10     2019-10        batiste                    26
11     2019-10        beautix                   321
12     2019-10    beauty-free                   137
13     2019-10  beautyblender                    17
14     2019-10     beauugreen                     6
15     2019-10         benovy                     6
16     2019-10        bergamo                    26
17     2019-10      bespecial                    18
18     2019-